# Ag-Analytics® - Elevation Index API 

Local elevation on a field can have a strong correlation with harvest yield. The correlation stems from the drainage dynamics that are present on a given field, with low areas being productive in dry years and high areas being productive in wet years. 

The Elevation Index API supports calculation of a set of elevation indices including field relative elevation, maximum slope, ,local Topographic Position Index(TPI), and local Topographic Roughness Index(TRI) for a given field elevation. 

### Required libraries

In [1]:
import requests
import json
import time
import os
from pandas.io.json import json_normalize
from collections import defaultdict
import pandas as pd
import zipfile, io
from IPython.display import Image

%matplotlib inline
%autosave 0

Autosave disabled


### Request Parameter Details

Request URL:  https://ag-analytics.azure-api.net/elevation-index

1). __raster__(string): Local path to the elevation raster the index will be calculated from
       
2). __index_list__(string): The elevation index of interest to retrieve.
    
    i. Relative_Elevation: Derived by calculating the z-score of every point in the field
    
    ii. Slop: The maximum slope of each point on the field is calculated by taking the gradient of the elevation for each point.
    
    iii.TPI: The topographic position index is a method of measuring the elevation of a given point with respect to a surrounding neighborhood of surrounding points. This index indicates the local extrema,or points of maximum and minimum elevation on a sub-field level. 
    
    iV.TRI:  The topographic Roughness index express the amount of elevation difference between adjacent points on local field

3). __Outside_Radius(optional)__: Outer radius of search area of each point. Used in TPI only.

        (Ex:Outside_Radius =11 the outer search radius is 11 * cellsize. When cellsize is 0.001 of the elevation raster, the radius is about 11 * 8 = 88 meter

4). __Inside_Radius(optional)__: Inner radius of search area. Used in TPI only.

5). __Search_Radius(optional)__: Radius of computational box. search_radius = 1 => 3x3 box, 2 => 5x5 box, etc. Used in TRI only



### API Function

In [8]:
def ElevationIndex(values, headers, files = None):
    try:
        url = 'https://ag-analytics.azure-api.net/elevation-index'
#         url = "http://127.0.0.1:5000/agflask/api/v1.0/elevation_index_service/elevation_index"
        
        if files == None:
            response = requests.post(url, json=values, headers=headers).json()
        else:
            response = requests.post(url, files = files, data = values).json()
        #resp = json.loads(response)
        print(response)
        
        return response
    
    except Exception as e:
        print(e)
        raise e

### Request with GeoJSON shape.

In [9]:
# Request as GeoJSON.
values = {'Shape': '{\"type\":\"Feature\",\"geometry\":{\"type\":\"Polygon\",\"coordinates\":[[[-76.5907145, 42.443918], [-76.5898132, 42.4224745], [-76.5699863, 42.4230447], [-76.5710592, 42.4443296], [-76.5907145, 42.443918]]]},\"properties\":{\"OBJECTID\":4944402,\"CALCACRES\":46.15999985,\"CALCACRES2\":null},\"id\":4944402}',
            'index_list':"['Relative_Elevation']",
            'Outside_Radius' :'5',
            'Inside_Radius':'3',
            'Search_Radius' :'2',
            'Legend_Range': '10'
}

headers={'Content-Type':'application/json'}


In [10]:
IndexResponse = ElevationIndex(values, headers)

{'statusCode': 500, 'message': 'Internal server error', 'activityId': '3d07382b-1ee8-4191-812e-f2827728ab52'}


### Request using GeoTIFF DEM Raster

In [38]:
#Path to raster in your local machine
rasterpath = r"C:\Users\Alex\Documents\result_raster_dem_20200612135704149641.tif"

files = [('elevation_raster',(open(rasterpath, 'rb')))]
print(files)

#Parameters to create index
values = {'index_list':"['Slope', 'Relative_Elevation', 'TPI', 'TRI']",
            'Outside_Radius' :'5',
            'Inside_Radius':'3',
            'Search_Radius' :'2',
            'Legend_Range': '10'
}

# Basic Header Pattern.
headers={'Content-Type':'multipart/form-data'}

# Header for using a subscription key.
# headers={'content-type': "application/json",'Ocp-Apim-Subscription-Key': "xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx"}

[('elevation_raster', <_io.BufferedReader name='C:\\Users\\Alex\\Documents\\result_raster_dem_20200612135704149641.tif'>)]


### Calling API Function and Displaying Response

In [39]:
IndexResponse = ElevationIndex(values, headers, files)

{'Relative_Elevation': 'raster_RelativeElevation_20200612150500199427.tif', 'Slope': 'raster_Slope_20200612150500199427.tif', 'TPI': 'raster_TPI_20200612150500199427.tif', 'TRI': 'raster_TRI_20200612150500199427.tif', 'features': [[{'attributes': {'CellSize': [9.259259269220297e-05, -9.259259269220297e-05], 'CoordinateSystem': 'GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433],AUTHORITY["EPSG","4326"]]', 'Extent': '-76.5907145, 42.42238515553195, -76.57006635182964, 42.4443296', 'Legend': [{'Area': '10.0 %', 'Count': 4906, 'CountAllPixels': 49060, 'Max': -1.4202146530151367, 'Mean': -2.046072244644165, 'Min': -2.6719298362731934, 'color': '#4d76d5'}, {'Area': '10.0 %', 'Count': 4906, 'CountAllPixels': 49060, 'Max': -0.9852170109748839, 'Mean': -1.2027158319950102, 'Min': -1.4202146530151367, 'color': '#738bbd'}, {'Area': '10.0 %', 'Count': 4906, 'CountAllPi

### Output Index Summary

In [6]:
df=defaultdict(list)
for value in IndexResponse['features']:
    attributes = value[0]['attributes']
    df['Index'].append(attributes['Product'])
    df['Max'].append(float(attributes['Max']))
    df['Mean'].append(float(attributes['Mean']))
    df['Min'].append(float(attributes['Min']))
    df['pngb64'].append(attributes['pngb64'])

indexdf=pd.DataFrame.from_dict(df)
#indexdf = indexdf.sort_values(by = ['Area'], ascending=False).style.applymap(lambda x:"background-color: %s"%x, subset=['Color'])
indexdf

,Index,Max,Mean,Min,pngb64
0,Relative_Elevation,1.495030,-0.000001,-2.671930,"data:image/png;base64, iVBORw0KGgoAAAANSUhEUgA..."
1,Slope,0.185605,0.022670,0.000024,"data:image/png;base64, iVBORw0KGgoAAAANSUhEUgA..."
2,TPI,4.567409,-0.020727,-6.094990,"data:image/png;base64, iVBORw0KGgoAAAANSUhEUgA..."
3,TRI,5.874935,1.215647,0.037480,"data:image/png;base64, iVBORw0KGgoAAAANSUhEUgA..."


### Displaying Index Image

In [7]:
#####################################
# Select one index to display. Must match one of the following: 
# ['Slope', 'Relative_Elevation', 'TPI', 'TRI']
Index = 'Slope'
######################################

In [8]:
#Select one index
Index = Index

#Image Legend
df=defaultdict(list)
for product in IndexResponse['features']:
    if Index == product[0]['attributes']['Product']:
        Legend = product[0]['attributes']['Legend']
for val in Legend:
    df['Max'].append(float(val['Max']))
    df['Mean'].append(float(val['Mean']))
    df['Min'].append(float(val['Min']))
    df['Color'].append(val['color'])
    df['Range'] = ['Low','Medium','High']
    
legenddf = pd.DataFrame.from_dict(df)    
legenddf.style.applymap(lambda x:"background-color: %s"%x, subset=['Color'])

ValueError: arrays must all be same length

In [9]:
IndexImage = indexdf.loc[indexdf['Index'] == Index,'pngb64'].iloc[0]
Image(url = IndexImage, width= 500, height =500)

# GET Request

In [10]:
# Specify local path where file will be downloaded.
local_path = r"C:\Users\Alex\Documents" # E.g., r"C:\Users\John_Doe\Documents\rasters"

# Create GET request payload.
Index = 'Relative_Elevation'
values = {'filenames': [IndexResponse[Index],]}
print(values)

{'filenames': ['raster_RelativeElevation_20200612135833933444.tif']}


In [11]:
def Elevation_Index_get(values, local_path):
    try:
        url = 'https://ag-analytics.azure-api.net/elevation-index'
    
        download_path = os.path.join(local_path, values['filenames'][0])
        print(values)
     
        response = requests.get(url, params=values)
        open(download_path, 'wb').write(response.content)
        
        print(response.url)

        return response
    
    except Exception as e:
        print(e)
        raise e

In [12]:
Elevation_Index_get(values, local_path)

{'filenames': ['raster_RelativeElevation_20200612135833933444.tif']}
https://ag-analytics.azure-api.net/elevation-index?filenames=raster_RelativeElevation_20200612135833933444.tif


<Response [200]>